In [1]:
import os, sys
%pwd

'/config/workspace/notebooks'

In [2]:
os.chdir("../")
%pwd

'/config/workspace'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
from wild_blueberry.constants import * 
from wild_blueberry.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
        

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [6]:
import os
import urllib.request as request
import zipfile
from wild_blueberry import logger
from wild_blueberry.utils.common import get_size

In [7]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-09-19 18:19:44,460: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-09-19 18:19:44,461: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-19 18:19:44,463: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-09-19 18:19:44,464: INFO: common: created directory at: artifacts]
[2023-09-19 18:19:44,465: INFO: common: created directory at: artifacts/data_ingestion]
[2023-09-19 18:19:45,350: INFO: 1543902713: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23895
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'
Content-Type: application/zip
ETag: "1e3b68bf084de6585c9a6f09d859783cf345ecefe446182392c32fd1f308cecc"
Last-Modified: Tue, 19 Sep 2023 12:49:45 GMT
Strict-Transport-Security: max-age=31557600
timing-allow-origin: https://github.com
X-Content-Type-Options: nosniff
X-Frame-Options: deny
x-github-tenant: 
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 76